In [1]:
import pandas as pd
import script
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Airtable

In [3]:
airtable = script.load_airtable(
    "./Transit Data Quality Issues-Open Issues by Distrcit (4).csv"
)

## Catastrophic

In [4]:
cat_df = script.load_catastrophic_errors(
    "./feed_info_expired_feeds_2023-12-28T17_11_45.147002Z.xlsx"
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
cat_df

,service_name,district,uri,date
4,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-27
8,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-26
13,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-25
17,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-24
21,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-23
25,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-22
29,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-21
33,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-20
37,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-19
41,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-12-18


In [6]:
script.summarize_cat(cat_df, airtable)

,service_name,uri,# of days with expired feed
0,Commerce Municipal Bus Lines,https://citycommbus.com/gtfs,2
1,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14


,service_name,uri,# of days with expired feed,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Commerce Municipal Bus Lines,https://citycommbus.com/gtfs,2,Commerce Schedule,Commerce Municipal Bus Lines,About to Expire Schedule Feed,The GTFS schedule feed is about to expire on 12/23,Yes
1,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14,Tracy Schedule,TRACER,Expired Schedule Feed,City of Tracy: GTFS Schedule Feed Expired,Yes


## Incompleteness

In [7]:
script.load_api_511("API511.xlsx")

These are Bay Area feeds that keep track of RT


,new
13,AC TRANSIT
22,Bay Area Rapid Transit
25,Caltrain
37,County Connection
40,Dumbarton Express Consortium
43,Emery Go-Round
46,FAST
52,Golden Gate Transit
55,Livermore Amador Valley Transit Authority
58,Marin Transit


In [8]:
airtable.loc[airtable.issue_type.str.contains('Completeness')]

,gtfs_datasets,services,issue_type,description,airtable_ticket
6,"B-Line Vehicle Positions,B-Line Trip Updates",B-Line,GTFS Realtime Completeness Problem,"Since November 19, there is a total stoppage in the production of GTFS data for B-Line Trip Updates and Vehicle Positions.",Yes
8,Bay Area 511 ACE TripUpdates,Altamont Corridor Express,GTFS Realtime Completeness Problem,"As of December 6, Bay Area 511 ACE GTFS-RT has recorded 17.5% in the past 14 days. Since December 6, TripUpdates has reported 0% trip data.",Yes
9,"Bay Area 511 Emery Go-Round VehiclePositions,Bay Area 511 Emery Go-Round TripUpdates",Emery Go-Round,GTFS Realtime Completeness Problem,"As of December 15, Bay Area 511 Emery Go-Round GTFS-RT has recorded 0% in the past 14 days. Since November 30, TripUpdates and VehiclePositions has reported 0% trip data.",Yes
10,"Bay Area 511 Fairfield and Suisun Transit Vehicle Positions,Bay Area 511 Fairfield and Suisun Transit TripUpdates",Fairfield and Suisun Transit,GTFS Realtime Completeness Problem,"From Nov 21 to Nov 28, there was a complete outage of GTFS-Realtime Trip Update and Vehicle Position data production.",Yes
11,Bay Area 511 Petaluma VehiclePositions,Petaluma Transit,GTFS Realtime Completeness Problem,"Vehicle Position real-time data production has been consistently inconsistent throughout the last month. The average of the last two weeks Vehicle Position data production is 34.63%, which is below the threshold.",Yes
12,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 37.21%, which is a reduction from 46.15% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 31.41% to 41.67%.\n\n",Yes
13,Bay Area 511 Vine Transit VehiclePositions,Vine Transit,GTFS Realtime Completeness Problem,GTFS-Realtime Vehicle Position data production has not been consistent throughout the month. The last two weeks' average Vehicle Position data production was 39.86%,Yes
34,"MVGO VehiclePositions,MVGO TripUpdates",MVGO,GTFS Realtime Completeness Problem,Since July 3 there have been zero trips accounted for in both the Vehicle Positions and Trip Updates for MVGO,Yes
36,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
37,"Montebello Vehicle Positions,Montebello TripUpdates",Montebello Bus Lines,GTFS Realtime Completeness Problem,"As of December 15, Montebello GTFS-RT has recorded 23.47% TripUpdates and 18.42% VehiclePositions in the past 14 days. Since December 11, TripUpdates and VehiclePosition has reported 0% trip data.",Yes


In [10]:
script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2023-12-28T17_05_01.563812Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2023-12-28T17_05_03.707126Z.xlsx",
    airtable,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Anaheim Resort Schedule,20.34,16.87,Anaheim Resort Schedule,Anaheim Resort Transportation,Service Accuracy,Anaheim Transportation Network: GTFS-Schedule issue: thousands of duplicate trips on Toy Story Line,Yes
1,Bay Area 511 ACE Schedule,18.75,OK,NA,NA,NA,NA,NA
2,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
3,Bay Area 511 BART Schedule,OK,0.00,NA,NA,NA,NA,NA
4,Bay Area 511 Commute.org Schedule,0.00,0.00,NA,NA,NA,NA,NA
5,Bay Area 511 Emery Go-Round Schedule,0.00,0.00,NA,NA,NA,NA,NA
6,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
7,Bay Area 511 MVGO Schedule,25.00,22.36,NA,NA,NA,NA,NA
8,Bay Area 511 Mission Bay Schedule,0.00,0.00,NA,NA,NA,NA,NA
9,Bay Area 511 Petaluma Schedule,12.23,0.00,NA,NA,NA,NA,NA
